## Training for the model.

TODO:
- Obtain data from alex and Kacper 
- Record a full conversation
- Train the model (using tensorflow)
- Test it in the main app

In [1]:
import os
import librosa
import numpy as np
from tqdm import tqdm
from pydub import AudioSegment

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

2024-07-17 13:11:11.093009: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-17 13:11:11.169127: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-17 13:11:11.169589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-17 13:11:11.294855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 13:11:12.301818: W tensorflow/compiler/tf

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from pydub import AudioSegment

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

# Constants
SR = 16000  # Sample rate
DURATION = 10  # Duration of each audio clip in seconds

# Function to load and preprocess audio file
def preprocess_audio(audio_file):
    # Load audio using pydub
    audio = AudioSegment.from_file(audio_file)

    # Resample audio to SR and convert to mono
    audio = audio.set_frame_rate(SR).set_channels(1)

    # Convert pydub.AudioSegment to numpy array
    audio_array = np.array(audio.get_array_of_samples(), dtype=np.float32)

    # Trim or pad audio to DURATION seconds
    target_length = int(SR * DURATION)
    if len(audio_array) < target_length:
        padding = target_length - len(audio_array)
        audio_array = np.pad(audio_array, (0, padding), mode='constant')
    elif len(audio_array) > target_length:
        audio_array = audio_array[:target_length]

    # Normalize audio
    audio_array /= np.max(np.abs(audio_array))

    return audio_array

# Function to extract features (e.g., spectrogram)
def extract_features(audio_array):
    # Compute spectrogram using librosa
    S = librosa.feature.melspectrogram(y=audio_array, sr=SR)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S

# Function to process audio files and create DataFrame
def process_files(mp3_files):
    data = {'audio': [], 'label': []}

    for mp3_file in tqdm(mp3_files):
        # Determine label from filename
        label = os.path.basename(mp3_file).split('_')[0]

        # Preprocess audio
        audio_array = preprocess_audio(mp3_file)

        # Extract features (e.g., spectrogram)
        features = extract_features(audio_array)

        # Add data to dictionary
        data['audio'].append(features)
        data['label'].append(label)

    # Convert data dictionary to DataFrame
    df = pd.DataFrame(data)
    
    return df

In [ ]:

directory = './training/'

# List all files in the directory
files = os.listdir(directory)

# Filter files with .mp3 extension
mp3_files = [file for file in files if file.endswith('.mp3')]


# Process files to create DataFrame
train_df = process_files(mp3_files)

#Save DataFrame to CSV file
train_df.to_csv('./training/trainining_audio_dataset.csv', index=False)

print("Audio preprocessing complete.")

In [ ]:
##Tensorflow training code (use train df)